In [2]:
import pandas as pd 
import numpy as np 
from matplotlib import pyplot as plt 
import seaborn as sns 
from ScrimLog import *

In [4]:
def update_FinalStat(save_dir='G:/공유 드라이브/NYXL Scrim Log/FinalStat/'):
    filepath = 'G:/공유 드라이브/NYXL Scrim Log/Csv/'
    filelist = os.listdir(filepath)
    csv_filelist = [x for x in filelist if x.endswith('.csv')]
    updated_csv = 'FilesUpdated.txt'

    f = open(filepath + updated_csv, 'r+')
    lines = f.readlines()
    updated_filelist = []

    for line in lines:
        updated_filelist.append(line.replace('\n', ''))

    csv_filelist_to_export = list(set(csv_filelist) - set(updated_filelist))

    # export to csv in FinalStat folder
    for filename in csv_filelist_to_export:
        ScrimLog(filename).export_to_csv()

        f.write(filename+'\n')
        print(f'File Exported: {filename}')

    f.close()

update_FinalStat()

In [6]:
# df_peth_FB = PETH(df_init=df_FinalStat, event_name='FinalBlows/s', period=5)
# df_peth_FB = df_peth_FB.get_PETH(of='Tracer').groupby(by=['MatchId', 'Map', 'Section', 'num_event', 'Timestamp', 'Team', 'Player', 'Hero']).sum()

# plt.figure(figsize=(20, 10))
# sns.lineplot(x='Timestamp', y='Cooldown1%', data=df_peth_FB, hue='Hero')
# plt.show()

# plt.figure(figsize=(20, 10))
# sns.lineplot(x='Timestamp', y='Cooldown2%', data=df_peth_FB, hue='Hero')
# plt.show()

# plt.figure(figsize=(20, 10))
# sns.lineplot(x='Timestamp', y='CooldownSecondaryFire%', data=df_peth_FB, hue='Hero')
# plt.show()

# plt.figure(figsize=(20, 10))
# sns.lineplot(x='Timestamp', y='CooldownCrouching%', data=df_peth_FB, hue='Hero')
# plt.show()

In [8]:
'''
PETH 개념을 거꾸로 가져가서 몇 가지 event가 동시에 or interval 이내에 일어났을 때를 기준으로 (t=0) 'deaths/s', 'final blows/s' 등의 event 확률 histogram 보여주는 기능 추가
'''

"\nPETH 개념을 거꾸로 가져가서 몇 가지 event가 동시에 or interval 이내에 일어났을 때를 기준으로 (t=0) 'deaths/s', 'final blows/s' 등의 event 확률 histogram 보여주는 기능 추가\n"